In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scripts import dl_utils, contour_gen
import h3
from tensorflow import keras

In [ ]:
rect_width = 0.005
start_date = '2021-01-01'
end_date = '2022-06-01'
mosaic_period = 1
spectrogram_interval = 1
coords = [66.226048, 56.651030]
site_id = h3.geo_to_h3(coords[1], coords[0], 15)
rect = dl_utils.rect_from_point(coords, rect_width)

In [ ]:
site = contour_gen.SiteContours(coords, site_id)
site.download_composites(start_date, end_date, mosaic_period, spectrogram_interval)
site.composites = site.data.composites

In [ ]:
from PIL import Image
# rescale image to be of a given size
def rescale_image(image, width, height):
    channels = image.shape[-1]
    rescaled = np.zeros((width, height, channels))
    image = (np.clip(image / 5000, 0, 1) * 255).astype(np.uint8)
    for i in range(channels):
        rescaled[:, :, i] = (np.array(Image.fromarray(image[:, :, i]).resize((height, width), resample=Image.BICUBIC)) / 255) * 5000
    return rescaled

def tile_image(image, tile_size):
    # function to break image into tiles
    # image is a 3D array of shape (height, width, channels)
    tile_width = tile_size[0]
    tile_height = tile_size[1]
    channels = image.shape[-1]
    tiles = np.zeros((image.shape[0] // tile_height, image.shape[1] // tile_width, tile_height, tile_width, channels))
    for i in range(tiles.shape[0]):
        for j in range(tiles.shape[1]):
            tiles[i, j, :, :, :] = image[i * tile_height:(i + 1) * tile_height, j * tile_width:(j + 1) * tile_width, :]
    num_img = tiles.shape[0] * tiles.shape[1]
    tiles = np.reshape(tiles, (num_img, tile_height, tile_width, channels))
    return tiles

def recombine_tiles(tiles):
    # function to recombine tiles into image
    # tiles is a 4D array of shape (num_tiles, tile_height, tile_width, channels)
    num_img = int(np.sqrt(tiles.shape[0]))
    tile_width, tile_height, channels = tiles.shape[1:]
    image = np.zeros((num_img * tile_height, num_img * tile_width, channels))
    counter = 0
    for i in range(num_img):
        for j in range(num_img):
            image[i * tile_height:(i + 1) * tile_height, j * tile_width:(j + 1) * tile_width, :] = tiles[counter, :, :, :]
            counter += 1
    return image

In [ ]:
model = keras.models.load_model('../../models/unet_48px_v0.1_2022-06-05.h5')
preds = []
scaled_composites = []
for image in site.composites:
    resample_scale = 8
    mask = rescale_image(image.mask.astype(np.uint8) * 255, 48 * resample_scale, 48 * resample_scale)
    mask = mask > 0
    image = rescale_image(image, 48 * resample_scale, 48 * resample_scale)
    masked_composite = np.ma.array(image, mask=mask.astype('bool'))
    scaled_composites.append(masked_composite)
    tiles = tile_image(masked_composite, (48, 48))
    tile_preds = model.predict(np.array([np.clip(tile / 3000, 0, 1) for tile in tiles]))
    recombined_preds = recombine_tiles(tile_preds)[:,:,1]
    recombined_preds = np.ma.array(recombined_preds, mask=mask[:,:,0])
    preds.append(recombined_preds)
    plt.imshow(image[:,:,3:0:-1] / 3000)
    plt.show()
site.scaled_composites = scaled_composites
site.preds = preds

In [ ]:
site.threshold_predictions(threshold=0.1)
site.mask_predictions(window_size=2)
site.generate_contours(threshold=0.5, scale=1, plot=True)
site.generate_polygons(plot=False)

In [ ]:
site.compile_contours()
site.contour_gdf

In [ ]:
name = 'test_contours'
site.contour_gdf.to_file(f'/Users/ckruse/Downloads/{name}.geojson', driver='GeoJSON')